**This Trial For Predictions for T20 Matches over wise using XGBOOST.

In [1]:
#Import all the packages
import operator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
import ast
import matplotlib.pyplot
%matplotlib inline
import seaborn as sns
from ipywidgets import interact
#from packages_stat import packages_stats

In [2]:
#Import match type 
pd.set_option('display.max_columns', 50)

match_df=pd.read_csv("real_final_scores_2.csv")

match_df = match_df[(match_df['info.match_type']=='ODI')]
#match_df = match_df[(match_df['info.outcome.result']!='no result') & (match_df['info.outcome.result']!='tie')]

In [3]:
#input the team names
team_1="India"
team_2="Sri Lanka"

In [ ]:
#get the data of those above mentioned matches in a dataframe.
def get_matches(match_df,team_1,team_2):
    match_df['info.teams'] = match_df['info.teams'].str.strip().apply(ast.literal_eval)
    ndf = match_df['info.teams'].apply(pd.Series)
    idx = ndf[ndf.isin([team_1,team_2]).all(1)].index
    match_df=match_df.loc[idx]
    return match_df
    
df=get_matches(match_df,team_1,team_2)

In [ ]:
#Encode the Batting 1st team as 1 and Batting 2nd as 0. 
match_index=df['index_all'].unique()
d=[]
for i in match_index:
    match=df[df['index_all']==i]
    try:
        for j,k in match.iterrows():
            a,b=match['team'].unique()
            if (k['team']==a):
                d.append(1)
            else:
                d.append(0)
    except:
        print(i)
            
           
df['team_order']=d
        

In [ ]:
#Find the runs and wickets per over for both the teams
df['wicket.player_out'].fillna(0,inplace=True)
df['wicket.player_out'].loc[df['wicket.player_out']!=0]=1
train_df=df.groupby(['index_all','over','team','info.outcome.winner','team_order'])[['runs.total','wicket.player_out']].agg(['sum']).reset_index()
train_df.columns= train_df.columns.get_level_values(0)


In [ ]:
#Find the Innings Score and wickets
train_df['innings_wickets']=train_df.groupby(['index_all','team'])['wicket.player_out'].cumsum()
train_df['innings_score']=train_df.groupby(['index_all','team'])['runs.total'].cumsum()

In [ ]:
# Find  the target column
temp_df=train_df.groupby(['index_all','team_order'])['runs.total'].sum().reset_index()
temp_df=temp_df.loc[temp_df['team_order']==1,:]
temp_df['team_order']=0
temp_df.columns = ['index_all','team_order','score_target']
train_df=train_df.merge(temp_df,how='left',on=['index_all','team_order'])
train_df['score_target']=train_df['score_target'].fillna(-1)

In [ ]:
#Find the remaining target of the innings
def get_remaining_target(row):
    if row['score_target'] == -1.:
        return -1
    else:
        return row['score_target'] - row['innings_score']

In [ ]:
train_df['remaining_target'] = train_df.apply(lambda row: get_remaining_target(row),axis=1)

In [ ]:
# Find the run rate of the innings
train_df['run_rate'] = train_df['innings_score'] / train_df['over']

In [ ]:
#Find the required run rate 
def get_required_runrate(row):
    if row['remaining_target'] == -1:
        return -1.
    elif row['over'] == 50:
        return 69
    else:
        return row['remaining_target'] / (50-row['over'])

In [ ]:
train_df['required_run_rate'] = train_df.apply(lambda row: get_required_runrate(row), axis=1)

In [ ]:
def get_runrate_diff(row):
    if row['team_order'] == 1:
        return -1
    else:
        return row['run_rate'] - row['required_run_rate']

In [ ]:
train_df['runrate_diff'] = train_df.apply(lambda row: get_runrate_diff(row), axis=1)

In [ ]:
train_df['target'] = (train_df['team'] == train_df['info.outcome.winner']).astype('int')

Now for Cross validation, We can use sklearn.cross_validation and use last match as test dataset. 

In [ ]:
#val is for testing
#dev is for training
#test_df=val_df[x_cols[:]]
#batting_1st=test_df['team']==1
#batting_2nd=(test_df.team==2)&(test_df.over<6)
#train_df_df=test_df[batting_1st | batting_2nd]

x_cols = ['team_order', 'over', 'runs.total', 'wicket.player_out', 'innings_wickets', 'innings_score', 'score_target', 'remaining_target', 'run_rate', 'required_run_rate', 'runrate_diff']
#train_df=train_df.drop(['info.outcome.winner'],axis=1)
val_df = train_df.loc[train_df.index_all == 905,:]
dev_df = train_df.loc[train_df.index_all != 905,:]

# create the input and target variables #
dev_X = np.array(dev_df[x_cols[:]])
dev_y = np.array(dev_df['target'])
val_X = np.array(val_df[x_cols[:]])[:-1,:]
val_y = np.array(val_df['target'])[:-1]
print(dev_X.shape, dev_y.shape)
print(val_X.shape, val_y.shape)

**We use XGBoost Algorithm, to predict outcome of the 2nd innings based on various factors such as
1. 1st Innings Score
2. Number of Wickets Fallen
3. Past History of Matches played between 2 teams
4. Over by Over comparision between number of wickets and run scored by each team. 
5. Current Run Rate and Required Run Rate

In [ ]:
def runXGB(train_X, train_y, seed_val=0):
    import xgboost as xg
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.05
    param['max_depth'] = 8
    param['silent'] = 1
    param['eval_metric'] = "auc"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = 100
    #plst = list(param.items())
    xgtrain = xg.DMatrix(train_X, label=train_y)
    model = xg.train(param, xgtrain, num_rounds)
    return model

In [ ]:
#Predict the outcome using XGboost Function
import xgboost as xgb
model=runXGB(dev_X, dev_y)
xgtest = xgb.DMatrix(val_X)
preds = model.predict(xgtest)

In [ ]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i,feat))
    outfile.close()

create_feature_map(x_cols)
importance = model.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()

# create a function for labeling #
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.02*height,
                '%f' % float(height),
                ha='center', va='bottom')
        
labels = np.array(imp_df.feature.values)
ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(12,6))
rects = ax.bar(ind, np.array(imp_df.fscore.values), width=width, color='y')
ax.set_xticks(ind+((width)/2.))
ax.set_xticklabels(labels, rotation='vertical')
ax.set_ylabel("Importance score")
ax.set_title("Variable importance")
autolabel(rects)
plt.show()

In [ ]:
val_df[x_cols[:]].columns

In [ ]:
out_df = pd.DataFrame({'Team1':val_df['team_order'].values})
#out_df['Team1']=out_df.sort_values(by='Team1')
#out_df['is_batting_team'] = val_df.team.values
out_df['innings_over']=val_df['over'].values
out_df['innings_score'] = val_df['innings_score'].values
out_df['innings_wickets'] = val_df['innings_wickets'].values
out_df['score_target'] = val_df['score_target'].values
out_df['total_runs'] = val_df['runs.total'].values
out_df['predictions'] = list(preds)+[1]
out_df.sort_values(['Team1','innings_over'],inplace=True)


In [ ]:
out_df[out_df['Team1']==0]['innings_over'].max()

In [ ]:
out_df[out_df['Team1']==1]['predictions'].sum()/out_df[out_df['Team1']==1]['innings_over'].max()
((out_df[out_df['Team1']==0]['predictions'].sum())-1)/out_df[out_df['Team1']==0]['innings_over'].max()

# prob_1=out_df[out_df['Team1']==1]['predictions'].sum()/out_df[out_df['Team1']==1]['innings_over'].max()
# prob_2=(out_df[out_df['Team1']==0]['predictions'].sum()-1)/out_df[out_df['Team1']==0]['innings_over'].max()
# print(prob_1)
# print(prob_2)


In [ ]:
out_df[out_df['Team1']==0]

In [ ]:
out_df[out_df['Team1']==0]['predictions'].max()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,6))
ax2 = ax1.twinx()
labels = np.array(out_df['innings_over'])
ind = np.arange(len(labels))
width = 0.7
rects = ax1.bar(ind, np.array(out_df['total_runs']), width=width, color=['green']*50 + ['yellow']*50)
ax1.set_xticks(ind+((width)/0.2))
ax1.set_xticklabels(labels, rotation='vertical')
ax1.set_ylabel("Runs in the given over")
ax1.set_xlabel("Innings and over")
ax1.set_title("Win percentage prediction for India vs Sri Lanka - over by over")

ax2.plot(ind+0.35, np.array(out_df['predictions']), color='b', marker='o')
ax2.set_ylabel("Win percentage", color='b')
ax2.set_ylim([0,1])
ax2.grid(b=False)
plt.show()